In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd

In [8]:
def scrapeItems(urlList, itemName):
    itemDf = pd.DataFrame(columns=['Brand', 'Price', 'Image', 'Height', 'Width', 'Depth', 'Seller', 'ProductLink'])

#     urlList = ["https://www.ikea.com/us/en/p/friheten-sleeper-sectional-3-seat-w-storage-skiftebo-blue-s69297560/", "https://www.ikea.com/us/en/p/karlstad-sofa-knisa-light-gray-s39132702/", "https://www.ikea.com/us/en/p/kivik-sofa-hillared-with-chaise-hillared-anthracite-s69193666/", "https://www.ikea.com/us/en/p/hemnes-daybed-with-3-drawers-2-mattresses-white-meistervik-firm-s59186119/", "https://www.ikea.com/us/en/p/knislinge-sofa-idhult-black-80278923/", "https://www.ikea.com/us/en/p/faerloev-sleeper-sofa-flodafors-white-s09252753/", "https://www.ikea.com/us/en/p/sandbacken-sleeper-sectional-3-seat-lofallet-beige-s89217906/"]
    for url in urlList:
        #url = 'https://www.ikea.com/us/en/p/landskrona-loveseat-gunnared-dark-gray-metal-s89270277/'
        response = requests.get(url)
        response
        soup = BeautifulSoup(response.text, "html.parser")
        brand = soup.findAll("span", {"class":"product-pip__name"})[0].contents[0]
        price = soup.findAll("span", {"class":"product-pip__price__value"})[0].contents[0]
        #Contains all dimension-names from product description
        dimensions_desc = soup.findAll("dt", {"class":"product-pip__definition-list-item"})

        imgLink = soup.findAll("img", {"class":""})[0]['src']
        #Contains all dimension-values from product description
        dimensions = soup.findAll("dd", {"class":"product-pip__definition-list-item"})
        dimensions

        #Finding All height-related dimensions
        height_index = []
        for dim in range(0, len(dimensions_desc)):
            if("height" in dimensions_desc[dim].contents[0].lower()):
                height_index.append(dim)

        #Finding max of given heights. Seemed in sync with most of the images 
        #with minute variations in the exceptions.
        heights = []
        for index in height_index:
            heights.append(dimensions[index].contents[0])

        height = max(heights)


        width_index = []
        for dim in range(0, len(dimensions_desc)):
            if("width" in dimensions_desc[dim].contents[0].lower()):
                width_index.append(dim)
        widths = []
        for index in width_index:
            widths.append(dimensions[index].contents[0])

        width = max(widths)


        depth_index = []
        for dim in range(0, len(dimensions_desc)):
            if("depth" in dimensions_desc[dim].contents[0].lower()):
                depth_index.append(dim)
        depths = []
        for index in depth_index:
            depths.append(dimensions[index].contents[0])

        depth = max(depths)
        depth

        productLink=url
        print(brand, price, imgLink, height, width, depth, productLink)
        itemDf = sofaDf.append({"Brand":brand, "Price":price, "Image":imgLink, "Height":height, "Width":width, "Depth":depth, "Seller":"IKEA", "ProductLink":productLink}, ignore_index=True)

    itemDf
    
    filename = itemName+".csv"
    itemDf.to_csv(filename)

In [10]:
fourProducts = pd.read_csv("FourProductsURL.csv")
noOfItems = len(fourProducts)
items = list(fourProducts)
for itemNumber in range(len(items)):
    scrapeItems(fourProducts[items[itemNumber]],items[itemNumber])

FRIHETEN $599.00 https://www.ikea.com/PIAimages/0690243_PE723167_S5.JPG?f=s 26 " 55 1/8 " 59 1/2 " https://www.ikea.com/us/en/p/friheten-sleeper-sectional-3-seat-w-storage-skiftebo-blue-s69297560/
KARLSTAD $399.00 https://www.ikea.com/PIAimages/0404895_PE577343_S5.JPG?f=s 31 1/2 " 80 3/4 " 36 5/8 " https://www.ikea.com/us/en/p/karlstad-sofa-knisa-light-gray-s39132702/
KIVIK $849.00 https://www.ikea.com/PIAimages/0479956_PE619108_S5.JPG?f=s 32 5/8 " 110 1/4 " 64 1/8 " https://www.ikea.com/us/en/p/kivik-sofa-hillared-with-chaise-hillared-anthracite-s69193666/
HEMNES $599.00 https://www.ikea.com/PIAimages/0636371_PE697851_S5.JPG?f=s 7 1/8 " 75 5/8 " 27 1/2 " https://www.ikea.com/us/en/p/hemnes-daybed-with-3-drawers-2-mattresses-white-meistervik-firm-s59186119/
KNISLINGE $349.00 https://www.ikea.com/PIAimages/0340648_PE528031_S5.JPG?f=s 37 " 80 3/4 " 35 " https://www.ikea.com/us/en/p/knislinge-sofa-idhult-black-80278923/
FÄRLÖV $999.00 https://www.ikea.com/PIAimages/0567578_PE667131_S5.JPG